In [1]:
import numpy as np

# 定义映射函数
def map_func(char):
    if char == ".":
        return 0
    elif char.isdigit():
        return int(char)
    elif char == "A":
        return 0
    else:
        raise ValueError(f"Invalid character {char}")

# 打开文件并读取每一行，并对每一行进行映射转换
with open("../map/1.txt", "r") as f:
    result = np.array([list(map(map_func, line.strip())) for line in f])

print(result)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 1 0 0]
 ...
 [0 0 1 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [2]:
x, y = result.nonzero()
print(x,y)

[ 2  2  2  2  2  2  2  2 20 31 31 31 31 36 49 49 49 62 67 67 67 67 78 97
 97 97 97 97 97 97 97] [ 2  6 10 44 55 89 93 97 49 18 26 73 81 49 34 49 64 49 18 26 73 81 49  2
  6 10 44 55 89 93 97]


In [3]:
xy = np.stack((x,y),-1)
XY = xy[:, None] - xy[None, :]
XY = np.linalg.norm(XY,2,-1)
print(XY)

[[  0.           4.           8.          42.          53.
   87.          91.          95.          50.32891813  33.12099032
   37.64306045  76.69419796  84.1546196   58.00862005  56.8594759
   66.46803743  77.80102827  76.21679605  66.94027188  69.28924881
   96.26006441 102.30347013  89.35882721  95.          95.08417324
   95.33624704 103.8701112  108.78419003 128.81770065 131.55227098
  134.35028843]
 [  4.           0.           4.          38.          49.
   83.          87.          91.          46.61544808  31.38470965
   35.22782991  73.00684899  80.41144197  54.81788029  54.70831747
   63.70243323  74.65252842  73.81734214  66.09841148  68.00735254
   93.34880824  99.24716621  87.32124598  95.08417324  95.
   95.08417324 102.31813134 106.89246933 126.15070353 128.81770065
  131.55227098]
 [  8.           4.           0.          34.          45.
   79.          83.          87.          42.95346319  30.08321791
   33.12099032  69.35416354  76.69419796  51.73973328  52.77309

In [5]:
def calculate_weights(adjacency_matrix):
    """
    计算每个节点的权重
    """
    return np.sum(adjacency_matrix, axis=1)

def assign_nodes_to_subgraphs(weights, node_types, adjacency_matrix):
    """
    将节点分配到四个子图中
    """
    # 初始化四个子图
    subgraphs = [[] for _ in range(4)]
    subgraph_weights = [0 for _ in range(4)]
    subgraph_node_types = [set() for _ in range(4)]
    
    # 将每个节点分配到最小权重的子图中
    for node_idx in np.argsort(weights):
        node_type = node_types[node_idx]
        # 计算每个子图与当前节点的连接权重和
        subgraph_weights_with_node = []
        for subgraph_idx in range(4):
            subgraph_weight_with_node = subgraph_weights[subgraph_idx] + np.sum(adjacency_matrix[node_idx][subgraphs[subgraph_idx]])
            subgraph_weights_with_node.append(subgraph_weight_with_node)
        # 选择权重和最小的子图并将当前节点加入该子图中
        min_weight_subgraph_idx = np.argmin(subgraph_weights_with_node)
        subgraphs[min_weight_subgraph_idx].append(node_idx)
        subgraph_weights[min_weight_subgraph_idx] = subgraph_weights_with_node[min_weight_subgraph_idx]
        subgraph_node_types[min_weight_subgraph_idx].add(node_type)
        
    return subgraphs, subgraph_node_types

def find_subgraphs(adjacency_matrix, node_types):
    """
    将图分割为四个子图，每个子图包含至少一种类型的节点，并且子图的权重和尽量小
    """
    weights = calculate_weights(adjacency_matrix)
    
    # 重复直到每个子图都包含至少一种类型的节点
    while True:
        subgraphs, subgraph_node_types = assign_nodes_to_subgraphs(weights, node_types, adjacency_matrix)
        if all(len(node_types_in_subgraph) > 0 for node_types_in_subgraph in subgraph_node_types):
            break
        
    return subgraphs


In [4]:
node_type = result[x,y]
node_type

array([1, 2, 4, 2, 3, 5, 3, 1, 6, 2, 5, 4, 3, 1, 7, 8, 7, 1, 2, 5, 4, 3,
       6, 1, 2, 4, 2, 3, 5, 3, 1])

In [11]:
subgraphs = find_subgraphs(XY, node_type)
for i, subgraph in enumerate(subgraphs):
    print(f"Subgraph {i}:")
    print(subgraph)
    print(set(node_type[subgraph]))

Subgraph 0:
[15, 14, 10, 9, 27, 28, 29, 30]
{1, 2, 3, 5, 7, 8}
Subgraph 1:
[17, 22, 20, 21, 26, 25, 24, 23]
{1, 2, 3, 4, 6}
Subgraph 2:
[13, 8, 11, 12, 3, 5, 6, 7]
{1, 2, 3, 4, 5, 6}
Subgraph 3:
[16, 19, 18, 4, 2, 1, 0]
{1, 2, 3, 4, 5, 7}


In [8]:
import numpy as np
from sklearn.cluster import KMeans

def spectral_clustering(adj_matrix, node_types):
    # Step 1: Construct Laplacian matrix
    D = np.diag(np.sum(adj_matrix, axis=1))
    L = D - adj_matrix

    # Step 2: Compute k smallest eigenvectors of L
    k = 4
    eigvals, eigvecs = np.linalg.eigh(L)
    idx = np.argsort(eigvals)[:k]
    X = eigvecs[:, idx]

    # Step 3: Normalize rows of X
    X_norm = np.linalg.norm(X, axis=1, keepdims=True)
    X_norm[X_norm == 0] = 1  # Avoid division by zero
    X_normalized = X / X_norm

    # Step 4: Apply K-Means clustering
    kmeans = KMeans(n_clusters=4).fit(X_normalized)

    # Step 5: Assign nodes to clusters
    clusters = [[] for _ in range(4)]
    for i, label in enumerate(kmeans.labels_):
        clusters[label].append(i)

    # Step 6: Ensure each cluster contains at least one node type
    for i in range(4):
        has_type = False
        for node in clusters[i]:
            if node_types[node] == 1:
                has_type = True
                break
        if not has_type:
            # Find nearest cluster with node type and move closest node
            nearest_cluster = None
            min_distance = np.inf
            for j in range(4):
                if j != i:
                    for node in clusters[j]:
                        if node_types[node] == 1:
                            distance = np.linalg.norm(X_normalized[node] - kmeans.cluster_centers_[i])
                            if distance < min_distance:
                                min_distance = distance
                                nearest_cluster = j
            node_to_move = min(clusters[nearest_cluster], key=lambda x: np.linalg.norm(X_normalized[x] - kmeans.cluster_centers_[i]))
            clusters[nearest_cluster].remove(node_to_move)
            clusters[i].append(node_to_move)

    # Compute weights of each cluster
    weights = [np.sum(adj_matrix[cluster][:, cluster]) for cluster in clusters]

    return clusters, weights


In [9]:
clusters, weights = spectral_clustering(XY, node_type)

In [10]:
print("Clusters:")
for i, cluster in enumerate(clusters):
    print(f"Cluster {i+1}: {cluster}")
print("Weights:", weights)

Clusters:
Cluster 1: [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Cluster 2: [14, 16, 8]
Cluster 3: [15, 30]
Cluster 4: [17]
Weights: [39922.283628188336, 190.59862173851607, 135.7645019878171, 0.0]


In [12]:
# 找到树的顶端节点
top_node = np.argmax(node_type == 7)

# 初始化已访问节点和未访问节点的列表
visited = [top_node]
unvisited = list(range(len(node_type)))
unvisited.remove(top_node)

# 初始化树的边列表
edges = []

# 遍历所有节点，构建树
while unvisited:
    min_weight = np.inf
    from_node = None
    to_node = None
    # 找到距离已访问节点最近的未访问节点，并构建边
    for i in visited:
        for j in unvisited:
            if XY[i, j] < min_weight:
                min_weight = XY[i, j]
                from_node = i
                to_node = j
    edges.append((from_node, to_node, min_weight))
    visited.append(to_node)
    unvisited.remove(to_node)

# 打印树的边列表
print("Edges of the tree:")
for edge in edges:
    print(edge)

Edges of the tree:
(14, 15, 15.0)
(15, 13, 13.0)
(15, 17, 13.0)
(15, 16, 15.0)
(13, 8, 16.0)
(17, 22, 16.0)
(8, 3, 18.681541692269406)
(3, 4, 11.0)
(22, 26, 19.6468827043885)
(26, 27, 11.0)
(14, 10, 19.697715603592208)
(10, 9, 8.0)
(14, 19, 19.697715603592208)
(19, 18, 8.0)
(16, 11, 20.12461179749811)
(11, 12, 8.0)
(16, 20, 20.12461179749811)
(20, 21, 8.0)
(9, 2, 30.083217912982647)
(2, 1, 4.0)
(1, 0, 4.0)
(12, 5, 30.083217912982647)
(5, 6, 4.0)
(6, 7, 4.0)
(18, 25, 31.04834939252005)
(25, 24, 4.0)
(24, 23, 4.0)
(21, 28, 31.04834939252005)
(28, 29, 4.0)
(29, 30, 4.0)


In [14]:
tier_1 = (7,8,9)
tier_2 = (4,5,6)
tier_3 = (1,2,3)

for tier in (tier_1, tier_2, tier_3):
    

Subgraphs:
Subgraph 1: [1 2 3 4 5 6 7 8]
Node types: [0 1 3 5 6 8]
Subgraph 2: [0 1 2 3 4 6 7 8]
Node types: [0 1 3 5 8]
Subgraph 3: [0 1 2 4 5 6 7 8]
Node types: [1 3 5 6 8]
Subgraph 4: [0 1 2 3 4 5 7 8]
Node types: [0 1 3 5 6]
